# Quick Start
Get up and running with NautilusTrader backtesting with some FX data. Exploring the Trade and Backtesting engine.

In [14]:
import datetime
import os
import shutil
from decimal import Decimal

import fsspec
import requests
import pandas as pd
from nautilus_trader.core.datetime import dt_to_unix_nanos
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.objects import Price, Quantity

from nautilus_trader.backtest.node import BacktestNode, BacktestVenueConfig, BacktestDataConfig, BacktestRunConfig, BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.persistence.wranglers import QuoteTickDataWrangler
from nautilus_trader.test_kit.providers import CSVTickDataLoader
from nautilus_trader.test_kit.providers import TestInstrumentProvider

# Grabbing Data

In [17]:
def load_fx_hist_data(filename: str, currency: str, catalog_path: str) -> None:
    instrument = TestInstrumentProvider.default_fx_ccy(currency)
    wrangler = QuoteTickDataWrangler(instrument)

    df = CSVTickDataLoader.load(filename, index_col=0, format="%Y%m%d %H%M%S%f")
    df.columns = ["bid_price", "ask_price", "size"]
    print(df)

    ticks = wrangler.process(df)

    catalog = ParquetDataCatalog(catalog_path)
    catalog.write_data([instrument])
    catalog.write_data(ticks)
    
def download(url: str) -> None:
    filename = url.rsplit("/", maxsplit=1)[1]
    with open(filename, "wb") as f:
        f.write(requests.get(url).content)


In [21]:
# Set data download directory
CATALOG_DIR = "./catalog"

# Download raw data
download(
    "https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/raw_data/fx_hist_data/DAT_ASCII_EURUSD_T_202001.csv.gz",
)

# Load data into CATALOG_DIR
load_fx_hist_data(
    filename="DAT_ASCII_EURUSD_T_202001.csv.gz",
    currency="EUR/USD",
    catalog_path=CATALOG_DIR,
)

/Users/ericli/anaconda3/envs/nt/lib/python3.12/site-packages/nautilus_trader/persistence/loaders.py:49: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


                         bid_price  ask_price  size
20200101 170000065                                 
2020-01-01 17:00:10.447    1.12120    1.12192     0
2020-01-01 17:00:10.498    1.12117    1.12161     0
2020-01-01 17:00:12.579    1.12120    1.12161     0
2020-01-01 17:00:12.630    1.12120    1.12172     0
2020-01-01 17:00:12.839    1.12120    1.12171     0
...                            ...        ...   ...
2020-01-31 16:59:53.963    1.10945    1.10961     0
2020-01-31 16:59:54.064    1.10945    1.10957     0
2020-01-31 16:59:54.711    1.10945    1.10960     0
2020-01-31 16:59:55.922    1.10945    1.10952     0
2020-01-31 16:59:56.198    1.10944    1.10952     0

[1637768 rows x 3 columns]


In [22]:
from nautilus_trader.persistence.catalog import ParquetDataCatalog
catalog = ParquetDataCatalog("./catalog")
catalog.instruments()

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

# Writing A Simple Trading Strategy

In [23]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model.data import QuoteTick
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.model.identifiers import InstrumentId
from nautilus_trader.model.objects import Quantity
from nautilus_trader.model.position import Position
from nautilus_trader.trading.strategy import Strategy, StrategyConfig


class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00010

    
class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period, slow_period=config.slow_period, price_type=PriceType.MID
        )
        # We copy some config values onto the class to make them easier to reference later on
        self.entry_threshold = config.entry_threshold
        self.instrument_id = config.instrument_id
        self.trade_size = Quantity.from_int(config.trade_size)
        self.entry_threshold = config.entry_threshold

        # Convenience
        self.position: Position | None = None

    def on_start(self):
        self.subscribe_quote_ticks(instrument_id=self.instrument_id)

    def on_stop(self):
        self.close_all_positions(self.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available.
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up
        
        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event: Event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT

            order = self.order_factory.market(
                instrument_id=self.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    def on_dispose(self):
        pass  # Do nothing else

# Configuring Backtests
Now that we have a trading strategy and data, we can configure our backtest to run. 

Nautilus uses a `BacktestNode` to orchestrate all backtest runs. This is necessary for the capabilties that Nautilus strives for. 

To configure a `BacktestNode` we first create an instance of a `BacktestRunConfig` to configure the minimal aspects of the backtest:
- `engine`: The engine for the backtest representing our core system, which will also contain our strategies
- `venues`: The simulated venues (exchanges or brokers) available in the backtest
- `data` : The input data we would like to perform the backtest on

## Venues
First, we create a venue configuration. For this example we will create a simulated FX ECN.
We pass "SIM" for its name parameter which acts as the Venue ID and other basic information. 

In [24]:
from nautilus_trader.config import BacktestVenueConfig

venue = BacktestVenueConfig(
    name="SIM",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"]
)

## Instruments
Second, we need to know about the instruments we would like to load the data for, we can use `ParquetDataCatalog` for this

In [25]:
instruments = catalog.instruments()
instruments

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

## Data
Third, we need to configure our data for the backtest. 
For each tick type (and instrument) we add a `BacktestDataConfig`. In this instance we are simply adding the `QuoteTick` for our EUR/USD instrument

In [26]:
from nautilus_trader.config import BacktestDataConfig
from nautilus_trader.model.data import QuoteTick

data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=QuoteTick,
    instrument_id=instruments[0].id,
    end_time="2020-01-10",
)

## Engine
Lastly, we need a `BacktestEngineConfig` which represents the configuration of our core trading system. Here we pass our trading strategies, and we can also adjust the log level and configure other components.

Strategies are added via the `ImportableStrategyConfig`, which allows importing strategies from arbitrary files or user packages.

In this instance our `MACDStrategy` is defined in the current module, which python refers to as `__main__` 

In [27]:
from nautilus_trader.config import BacktestEngineConfig
from nautilus_trader.config import ImportableStrategyConfig

# NautilusTrader currently exceeds the rate limit for Jupyter notebook logging (stdout output),
# this is why the `log_level` is set to "ERROR". If you lower this level to see
# more logging then the notebook will hang during cell execution. A fix is currently
# being investigated which involves either raising the configured rate limits for
# Jupyter, or throttling the log flushing from Nautilus.
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config=dict(
              instrument_id=instruments[0].id,
              fast_period=12,
              slow_period=26,
            ),
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),
)

# Running a Backtest
We can now pass various config pieces to the `BacktestRunConfig`, where this object now contains the full configuration for our backtest.

In [28]:
from nautilus_trader.config import BacktestRunConfig

config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
)

The `BacktestNode` class orchestrates the backtest run. The main reason for this separation of config and execution is that `BacktestNode` allows running multiple configurations (i.e. different parameters or batches of data). We are now ready to run some backtests!

In [29]:
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.results import BacktestResult

node = BacktestNode(configs=[config])

 # Runs one or many configs synchronously
results: list[BacktestResult] = node.run()

Now that the run is complete, we can also directly query for the BacktestEngine(s) used internally by the BacktestNode by using the run configs ID.

The engine(s) can provide additional reports and information.

In [30]:
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model.identifiers import Venue

engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,contingency_type,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20200103-1000-001-000-1,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-001,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-001,MARKET,BUY,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-03 10:00:00.242000+00:00,2020-01-03 10:00:00.242000+00:00
O-20200103-1000-001-000-2,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-002,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-002,MARKET,SELL,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-03 10:00:04.136000+00:00,2020-01-03 10:00:04.136000+00:00
O-20200103-1000-001-000-3,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-003,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-003,MARKET,SELL,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-03 10:00:06.093000+00:00,2020-01-03 10:00:06.093000+00:00
O-20200103-1000-001-000-4,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-004,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-004,MARKET,BUY,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-03 10:00:07.548000+00:00,2020-01-03 10:00:07.548000+00:00
O-20200103-1652-001-000-5,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-005,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-005,MARKET,SELL,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-03 16:52:30.901000+00:00,2020-01-03 16:52:30.901000+00:00
O-20200103-1652-001-000-6,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-006,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-006,MARKET,BUY,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-03 16:52:46.001000+00:00,2020-01-03 16:52:46.001000+00:00
O-20200105-1700-001-000-7,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-007,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-007,MARKET,BUY,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-05 17:00:32.758000+00:00,2020-01-05 17:00:32.758000+00:00
O-20200105-1705-001-000-8,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-008,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-008,MARKET,SELL,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-05 17:05:30.628000+00:00,2020-01-05 17:05:30.628000+00:00
O-20200108-0313-001-000-10,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-010,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-010,MARKET,BUY,1000000,...,NO_CONTINGENCY,None,None,None,None,None,None,None,2020-01-08 03:13:16.947000+00:00,2020-01-08 03:13:16.947000+00:00


In [31]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,peak_qty,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,
EUR/USD.SIM-MACDStrategy-000-d26a3c02-7c28-4952-8959-1927422947e3,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-1000-001-000-1,O-20200103-1000-001-000-2,BUY,FLAT,1000000,2020-01-03 10:00:00.242000+00:00,1578045604136000000,2020-01-03 10:00:04.136000+00:00,3894000000,1.11644,1.11635,['44.66 USD'],-8e-05,-134.66 USD
EUR/USD.SIM-MACDStrategy-000-7ff0e950-d14e-4bc1-b8d5-ce78f4fdf0ce,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-1000-001-000-3,O-20200103-1000-001-000-4,SELL,FLAT,1000000,2020-01-03 10:00:06.093000+00:00,1578045607548000000,2020-01-03 10:00:07.548000+00:00,1455000000,1.11604,1.11637,['44.65 USD'],-0.0003,-374.65 USD
EUR/USD.SIM-MACDStrategy-000-3116b7f0-cd8b-496c-aaa7-305f32eb51e1,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-1652-001-000-5,O-20200103-1652-001-000-6,SELL,FLAT,1000000,2020-01-03 16:52:30.901000+00:00,1578070366001000000,2020-01-03 16:52:46.001000+00:00,15100000000,1.11534,1.11543,['44.62 USD'],-8e-05,-134.62 USD
EUR/USD.SIM-MACDStrategy-000-f72aa9a9-b0c8-406f-9b0a-9c99632a3049,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200105-1700-001-000-7,O-20200105-1705-001-000-8,BUY,FLAT,1000000,2020-01-05 17:00:32.758000+00:00,1578243930628000000,2020-01-05 17:05:30.628000+00:00,297870000000,1.11703,1.11652,['44.67 USD'],-0.00046,-554.67 USD
EUR/USD.SIM-MACDStrategy-000-ca91e172-145b-486a-9986-47c862631f20,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-0313-001-000-9,O-20200108-0313-001-000-10,SELL,FLAT,1000000,2020-01-08 03:13:12.834000+00:00,1578453196947000000,2020-01-08 03:13:16.947000+00:00,4113000000,1.11403,1.1136,['44.55 USD'],0.00039,385.45 USD
EUR/USD.SIM-MACDStrategy-000-fae718a2-bfd8-485f-80e7-de699f7cafa8,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-0621-001-000-11,O-20200108-0621-001-000-12,SELL,FLAT,1000000,2020-01-08 06:21:39.588000+00:00,1578464517525000000,2020-01-08 06:21:57.525000+00:00,17937000000,1.11231,1.11211,['44.49 USD'],0.00018,155.51 USD
EUR/USD.SIM-MACDStrategy-000-89a976b5-19b5-4027-900e-ac3614f9be2b,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-0815-001-000-13,O-20200108-0815-001-000-14,SELL,FLAT,1000000,2020-01-08 08:15:01.502000+00:00,1578471314869000000,2020-01-08 08:15:14.869000+00:00,13367000000,1.11121,1.1112,['44.44 USD'],1e-05,-34.44 USD
EUR/USD.SIM-MACDStrategy-000,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-1726-001-000-15,O-20200108-1726-001-000-16,BUY,FLAT,1000000,2020-01-08 17:26:44.662000+00:00,1578504410316000000,2020-01-08 17:26:50.316000+00:00,5654000000,1.11164,1.11121,['44.45 USD'],-0.00039,-474.45 USD


In [32]:
engine.trader.generate_account_report(Venue("SIM"))

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-01-01 17:00:10.447000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,USD,[],True,{}
2020-01-03 10:00:00.242000+00:00,999977.67,0.00,999977.67,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:00.242000+00:00,999977.67,33515.53,966462.14,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:04.136000+00:00,999865.34,33515.53,966349.81,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:04.136000+00:00,999865.34,0.00,999865.34,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:06.093000+00:00,999843.02,0.00,999843.02,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:06.093000+00:00,999843.02,33503.52,966339.50,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:07.548000+00:00,999490.69,33503.52,965987.17,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:07.548000+00:00,999490.69,0.00,999490.69,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 16:52:30.901000+00:00,999468.38,0.00,999468.38,USD,SIM-001,MARGIN,USD,[],False,{}


In [33]:
results

[BacktestResult(trader_id='BACKTESTER-001', machine_id='Erics-MBP-3', run_config_id='1c083ef92d3738a56df36ce64ef293e2b31af16735c56f176f89b6cdeee24fa2', instance_id='7c5c44cb-c9ea-4398-98f0-2f2cf3d9942c', run_id='97457b22-59e6-46e7-a980-f4d055e34aec', run_started=1705464360734263000, run_finished=1705464363692504000, backtest_start=1577898010447000000, backtest_end=1578614399448000000, elapsed_time=716389.001, iterations=0, total_events=32, total_orders=16, total_positions=1, stats_pnls={'USD': {'PnL (total)': -1166.53, 'PnL% (total)': -0.1166530000000028, 'Max Winner': 385.45, 'Avg Winner': 270.48, 'Min Winner': 155.51, 'Min Loser': -34.44, 'Avg Loser': -284.5816666666667, 'Max Loser': -554.67, 'Expectancy': -145.81625000000003, 'Win Rate': 0.25}}, stats_returns={'Returns Volatility (252 days)': 0.00428839115091965, 'Average (Return)': -9.069839039973923e-05, 'Average Loss (Return)': -0.0002600756652692315, 'Average Win (Return)': 0.00019159706771608117, 'Sharpe Ratio (252 days)': -7.1